# Initial Imports

In [ ]:
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Since GitHub wouldn't let us upload all of the CSVs once we unarchived the zip file, we had to use GitHub Large File Storage and we opted to extract the files at the beginning.

In [ ]:
from zipfile import ZipFile
zip = ZipFile('Resources/archive.zip')
zip.extractall('Resources')

Here are the files in the `Resources` folder. Our analysis will start with the Mutual Fund files.

In [ ]:
import os

os.listdir("Resources")

This will show all columns in each of the DataFrames to help us make some determinations in which columns we will keep and / or lose.

In [ ]:
#Display all the columns (to see which to drop)
pd.set_option("display.max_columns", None)

## Read In CSV data for Mutual Fund prices A-Z


Since the CSV files came in 4 pieces, we will read them individually into DataFrames, then combine at the end.

In [ ]:
#Read in CSV data 
#MutualFund prices A-E
df_AE= pd.read_csv(
    Path("Resources/MutualFund Prices - A-E.csv")
)
df_AE.head()

In [ ]:
#Read in CSV data 
#MutualFund prices F-K
df_FK= pd.read_csv(
    Path("Resources/MutualFund Prices - F-K.csv")
)
df_FK.head()

In [ ]:
#Read in CSV data 
#MutualFund prices L-P
df_LP= pd.read_csv(
    Path("Resources/MutualFund Prices - L-P.csv")
)
df_LP.head()

In [ ]:
#Read in CSV data 
#MutualFund prices Q-Z
df_QZ= pd.read_csv(
    Path("Resources/MutualFund Prices - Q-Z.csv")
)
df_QZ.head()

In [ ]:
#Concatonate dataframes
mutual_fund_df= pd.concat([df_AE,df_FK,df_LP,df_QZ])
mutual_fund_df.shape 

In [ ]:
mutual_fund_df.dtypes

In [ ]:
#Convert to date
mutual_fund_df["price_date"]=pd.to_datetime(mutual_fund_df['price_date'])

In [ ]:
mutual_fund_df.dtypes

In [ ]:
mutual_fund_df.dropna().shape

As you can see, there are no NULLs in the `mutual_fund_df` DataFrame.

# Read in Mutual Fund informaiton csv


In [ ]:
#Read in CSV data 
#MutualFunds
mutualFunds= pd.read_csv(
    Path("Resources/MutualFunds.csv"),
    index_col="fund_symbol")

mutualFunds.head()

In [ ]:
# Find columns with only 1 value to drop
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

There are 3 columns currently listed as objects that are actually dates. We need to convert those into dates.

In [ ]:
#convert dates to date
mutualFunds["management_start_date"] = pd.to_datetime(mutualFunds["management_start_date"])
mutualFunds["inception_date"] = pd.to_datetime(mutualFunds["inception_date"])
mutualFunds["returns_as_of_date"] = pd.to_datetime(mutualFunds["returns_as_of_date"])

We also noticed that there are 6 columns that have only 1 unique value. We will remove those.

In [ ]:
mutualFunds_counts= mutualFunds.loc[: ,mutualFunds.dtypes=="object"].nunique()
mutualFunds_counts_one= mutualFunds_counts[mutualFunds_counts == 1].index.to_list()
print(mutualFunds_counts_one)

In [ ]:
#Drop columns with only 1 unique value
mutualFunds.drop(columns=mutualFunds_counts_one,inplace=True)
mutualFunds

_We noticed that there are many `NaN` values throughout the data set. We will now explore which columns/rows have NaN and will remove an appropriate amount of columns/rows._

In [ ]:
mutualFunds.dropna(axis = 0)

If we drop rows that have an `NaN`, then all data is removed. Instead, let's look at the columns that have missing data.

In [ ]:
# Looking at each column and the count of NaN in each column
list = []
for index, row in pd.DataFrame(mutualFunds.isna().sum()).iterrows():
    list.append((index,row))


In [ ]:
print(list[0])

We plan to find the number of columns that have above a certain percentage of `NaN` and then remove those columns. We have chosen 60% so far.

In [ ]:
threshold = 0.6*len(mutualFunds)
drop_columns = []
for i in list:
    if i[1][0] >= threshold:
        drop_columns.append(i[0])

In [ ]:
print(len(drop_columns))
drop_columns

In [ ]:
mutualFunds.drop(columns=drop_columns, inplace = True)
mutualFunds

In [ ]:
# Find columns with too many unique values, and to drop them
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

In [ ]:
mutualFunds[["fund_short_name", "fund_long_name","management_name", "management_bio", "investment_strategy", "top10_holdings"]]

All of those 6 columns are long text values. We've decided to remove them.

In [ ]:
#Drop unnecessary columns
mutualFunds.drop(columns=["fund_short_name", "fund_long_name","management_name", "management_bio", "investment_strategy", "top10_holdings"],inplace=True)

In [ ]:
mutualFunds

We now remove the numerical columns that have some type of redundant data. This includes quarterly data, and any analytical data. The intent here is to remove as many columns as possible since we currently have over 23K rows of data. If the model does not perform as well as we might hope, then we can return and keep some (if not all) of these columns.

In [ ]:
#Remove quarterly data
mutualFunds.drop(columns=["fund_return_2021_q1",
    "fund_return_2020_q4","fund_return_2020_q3","fund_return_2020_q2","fund_return_2020_q1",
    "fund_return_2019_q4","fund_return_2019_q3","fund_return_2019_q2","fund_return_2019_q1",
    "fund_return_2018_q4","fund_return_2018_q3","fund_return_2018_q2","fund_return_2018_q1",
    "fund_return_2017_q4","fund_return_2017_q3","fund_return_2017_q2","fund_return_2017_q1",
    "fund_return_2016_q4","fund_return_2016_q3","fund_return_2016_q2","fund_return_2016_q1",
    "fund_return_2015_q4","fund_return_2015_q3","fund_return_2015_q2","fund_return_2015_q1",
    "fund_return_2014_q4","fund_return_2014_q3","fund_return_2014_q2","fund_return_2014_q1",
    "fund_return_2013_q4","fund_return_2013_q3","fund_return_2013_q2","fund_return_2013_q1",
    "fund_return_2012_q4","fund_return_2012_q3","fund_return_2012_q2","fund_return_2012_q1",
    "fund_return_2011_q4","fund_return_2011_q3","fund_return_2011_q2","fund_return_2011_q1",
    "fund_return_2010_q4","fund_return_2010_q3","fund_return_2010_q2","fund_return_2010_q1",
    "fund_return_2009_q4","fund_return_2009_q3","fund_return_2009_q2","fund_return_2009_q1",
    "fund_return_2008_q3","fund_return_2008_q2","fund_return_2008_q1",
    "fund_alpha_3years","fund_beta_3years","fund_mean_annual_return_3years","fund_r_squared_3years","fund_stdev_3years","fund_sharpe_ratio_3years","fund_treynor_ratio_3years",
    "fund_alpha_5years","fund_beta_5years","fund_mean_annual_return_5years","fund_r_squared_5years","fund_stdev_5years","fund_sharpe_ratio_5years","fund_treynor_ratio_5years",
    "fund_alpha_10years","fund_beta_10years","fund_mean_annual_return_10years","fund_r_squared_10years","fund_stdev_10years","fund_sharpe_ratio_10years","fund_treynor_ratio_10years",
    "fund_return_category_rank_ytd","fund_return_category_rank_1month","fund_return_category_rank_3months","fund_return_category_rank_1year","fund_return_category_rank_3years",
    "fund_return_category_rank_5years","load_adj_return_1year","load_adj_return_3years","load_adj_return_5years","load_adj_return_10years"],
    inplace=True)

In [ ]:
#Remove metrics that we can caluclate ourselves
#keep: "esg_score","environment_score", "sustainability_score", "sustainability_rank",   "social_score",  "governance_score", 
mutualFunds.drop(columns=["esg_peer_count","peer_esg_min", "peer_esg_avg", "peer_esg_max",
    "peer_environment_min", "peer_environment_avg", "peer_environment_max", 
    "peer_social_min", "peer_social_avg", "peer_social_max",
    "peer_governance_min", "peer_governance_avg", "peer_governance_max"],
    inplace=True)


In [ ]:
mutualFunds.loc[: , mutualFunds.dtypes== "object"].nunique()

In [ ]:
print(f"Current DataFrame shape: {mutualFunds.shape}")
print(f"If we dropped NAs now DataFrame shape: {mutualFunds.dropna().shape}")

If we were to remove all `NULL`s at this point, there would be very few rows remaining. For numerical columns, we will replace the `NULL`s will the overall columns' average.

In [ ]:
# Calculate the average for each column
column_means = mutualFunds.mean(skipna=True, numeric_only=True)
# Replace NaN values in each column with the respective column average
mutualFunds.fillna(column_means, inplace=True)

In [ ]:
mutualFunds.dropna(inplace= True)
mutualFunds.shape

By replacing the `NULL`s with the column means and then dropping `NULL`s, we have keep over 50% of the rows and all of the columns.

# Binning!

In [ ]:
mutualFunds.loc[:,mutualFunds.dtypes == "object"].nunique()

Binning `fund_category`. It currently has 91 unique values and here are the value counts.

In [ ]:
mutualFunds["fund_category"].value_counts()

In [ ]:
fund_category_type_count = mutualFunds["fund_category"].value_counts()[mutualFunds["fund_category"].value_counts() > 150]
print(100*fund_category_type_count.sum()/len(mutualFunds))
print(fund_category_type_count.count())

By setting the cut-off value at 150, we only have 31 unique values and will keep ~85% of the values before changing the rest to "Other".

In [ ]:
for cat in mutualFunds["fund_category"]:
    if cat in fund_category_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_category"] = mutualFunds["fund_category"].replace(cat, "Other")

In [ ]:
mutualFunds["fund_category"].value_counts()

Binning `fund_family`. It currently has 310 unique values and here are the value counts.

In [ ]:
mutualFunds["fund_family"].value_counts()

In [ ]:
fund_family_type_count = mutualFunds["fund_family"].value_counts()[mutualFunds["fund_family"].value_counts() > 150]
print(100*fund_family_type_count.sum()/len(mutualFunds))
print(fund_family_type_count.count())

By setting the cut-off value at 150, we only have 19 unique values and will keep ~64% of the values before changing the rest to "Other".

In [ ]:
for cat in mutualFunds["fund_family"]:
    if cat in fund_family_type_count.index.to_list():
        next
    else:
        mutualFunds["fund_family"] = mutualFunds["fund_family"].replace(cat, "Other")

In [ ]:
mutualFunds["fund_family"].value_counts()

The columns `investment_type` and `size_type` both have 3 unique values. We will keep those as is and not bin them.

Binning `esg_peer_group`. It currently has 310 unique values and here are the value counts.

In [ ]:
mutualFunds["esg_peer_group"].value_counts()

In [ ]:
#finding threshhold to bin esg_peer_group
esg_peer_type_count = mutualFunds["esg_peer_group"].value_counts()[mutualFunds["esg_peer_group"].value_counts() > 100]
print(100* esg_peer_type_count.sum()/len(mutualFunds))
print(esg_peer_type_count.count())

By setting the cut-off value at 100, we only have 36 unique values and will keep ~89% of the values before changing the rest to "Other".

In [ ]:
#check the bins of esg_peer_group
esg_peer_type_count.index.to_list()
#replace bins in dataframe for esg_peer_group
for esg in mutualFunds["esg_peer_group"]:
    if esg in esg_peer_type_count.index.to_list():
        next
    else:
        mutualFunds["esg_peer_group"] = mutualFunds["esg_peer_group"].replace(esg, "Other")

In [ ]:
mutualFunds["esg_peer_group"].value_counts()